In [1]:
from nnunet_ext.network_architecture.nca.NCA2D import NCA2D
import torch
import torch.nn.functional as F

In [2]:
device = "cuda"

In [3]:
nca1 = NCA2D(num_channels=16, num_input_channels=1, num_classes=16, hidden_size=32, fire_rate=1, num_steps=10)
nca1.to(device)

NCA2D(
  (fc0): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  (fc1): Conv2d(32, 15, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=same, groups=16, padding_mode=reflect)
  (batch_norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
)

In [4]:
x = torch.randn(1,1,128,128, device=device)

In [5]:
before = torch.cuda.memory.memory_allocated()
state = nca1.make_state(x)
out = nca1.forward_internal(state)
out.sum().backward()
after = torch.cuda.memory.memory_allocated()
print(f"{(after-before) / 1024**2:.2f} MB allocated during baseline training step")

2.01 MB allocated during baseline training step


In [6]:
hidden_size2 = 4* 3
nca2 = NCA2D(num_channels=3, num_input_channels=0, num_classes=16, hidden_size=hidden_size2, fire_rate=1, num_steps=10)
nca2.fc0 = torch.nn.Conv2d(2 * (nca1.num_channels + nca2.num_channels), hidden_size2, kernel_size=1)
nca2.to(device)

NCA2D(
  (fc0): Conv2d(38, 12, kernel_size=(1, 1), stride=(1, 1))
  (fc1): Conv2d(12, 3, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (conv): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=same, groups=3, padding_mode=reflect)
  (batch_norm): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
)

In [7]:
nca1.requires_grad_(True);

In [8]:
before = torch.cuda.memory.memory_allocated()
state1 = torch.zeros(x.shape[0], nca1.num_channels - nca1.num_input_channels,
                    x.shape[2], x.shape[3], device=x.device)
state2 = torch.zeros(x.shape[0], nca2.num_channels, x.shape[2], x.shape[3], device=x.device)
state = [torch.cat([x, state1], dim=1), state2]
[s.shape for s in state]

delta_state0 = nca1.conv(state[0])
delta_state1 = nca2.conv(state[1])
delta_state0 = torch.cat([state[0], delta_state0], dim=1)
delta_state1 = torch.cat([delta_state0, state[1], delta_state1], dim=1)
delta_state0 = nca1.fc0(delta_state0)
delta_state1 = nca2.fc0(delta_state1)
delta_state0 = nca1.batch_norm(delta_state0)
delta_state1 = nca2.batch_norm(delta_state1)
delta_state0 = F.relu(delta_state0, inplace=True)
delta_state1 = F.relu(delta_state1, inplace=True)
delta_state0 = nca1.fc1(delta_state0)
delta_state1 = nca2.fc1(delta_state1)
temp_state0 = state[0][:, nca1.num_input_channels:] + delta_state0
temp_state0 = torch.cat([state[0][:, :nca1.num_input_channels], temp_state0], dim=1)
state[0] = temp_state0
state[1] = state[1] + delta_state1
[s.shape for s in state]
(state[0].sum() + state[1].sum()).backward()
after = torch.cuda.memory.memory_allocated()
print(f"{(after-before) / 1024**2:.2f} MB allocated during baseline training step")

2.44 MB allocated during baseline training step


In [9]:
delta_state1.shape

torch.Size([1, 3, 128, 128])